<a href="https://colab.research.google.com/github/rdkdaniel/Quantum-Computers-and-ML-DL/blob/main/Quantum_Computers_and_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A General Review**

**Introduction**

Interesting Reads and Resources:


*   https://www.nature.com/articles/s41467-021-22539-9
*   https://ai.googleblog.com/2021/06/quantum-machine-learning-and-power-of.html
*   https://blog.paperspace.com/beginners-guide-to-quantum-machine-learning/
*   https://research.ibm.com/topics/quantum-machine-learning
*   https://www.nature.com/articles/s41567-021-01287-z



In [ ]:
#Quantum advantage over a classical computer is often framed in terms of computational complexity classes. 
#Examples such as factoring large numbers and simulating quantum systems are classified as bounded quantum polynomial time (BQP) problems.
#Which are those thought to be handled more easily by quantum computers than by classical systems. 
#Problems easily solved on classical computers are called bounded probabilistic polynomial (BPP) problems

In [ ]:
#Google Research show:
#Learning algorithms equipped with data from a quantum process, such as a natural process like fusion or chemical reactions, form a new class of problems (which they call BPP/Samp).
#That efficiently perform some tasks that traditional algorithms without data cannot, and is a subclass of the problems efficiently solvable with polynomial sized advice (P/poly). 
#This demonstrates that for some machine learning tasks, understanding the quantum advantage requires examination of available data as well.

In [ ]:
#Checkout the image showing the intersection of BQP, BPP, BPP/Samp and P/Poly